In [23]:
import pandas as pd
import mysql.connector

In [24]:
# Generamos una función que nos permita crear una base de datos:

def make_ddbb(name_ddbb):
  
    mydb = mysql.connector.connect(
      host='127.0.0.1',
      user='root',
      password='AlumnaAdalab')

    mc = mydb.cursor()

    try:
        mc.execute(f"CREATE DATABASE IF NOT EXISTS {name_ddbb};")
        print(mc)
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [25]:
db_tiburones = make_ddbb('tiburones')

MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [26]:
# Generamos una segunda función para crear las tablas mediante una query.

def insert_table(nombre_bbdd, query):
    
    cnx = mysql.connector.connect(user='root', password=f"AlumnaAdalab",
                                     host='127.0.0.1', database=f"{nombre_bbdd}")
    
    mycursor = cnx.cursor()
 
    try: 
        mycursor.execute(query)
        cnx.commit() 
    # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [27]:
df_ataques = pd.read_csv('datos/ETL-2.csv', index_col=0)

In [28]:
attacks = ''' 
    CREATE TABLE IF NOT EXISTS `attacks` (
    `id_attacks` INT NOT NULL AUTO_INCREMENT,
    `id_climate` INT NOT NULL,
    `year` INT DEFAULT NULL,
    `type` VARCHAR(50) DEFAULT NULL, 
    `country` VARCHAR(50) DEFAULT NULL,  
    `age` FLOAT DEFAULT NULL,
    `species` VARCHAR(50) DEFAULT NULL,
    `date` VARCHAR (10) DEFAULT NULL,
    `fatal` CHAR (1) DEFAULT NULL,
    `sex` CHAR (1) DEFAULT NULL,
    PRIMARY KEY (`id_attacks`),
    INDEX `fk_attacks_climate` (`id_climate` ASC),
    CONSTRAINT `fk_attacks_climate`
    FOREIGN KEY (`id_climate`)
    REFERENCES `tiburones`.`climate` (`id_climate`)
    )
    
    ENGINE = InnoDB;
    '''

In [29]:
climate = '''
       CREATE TABLE IF NOT EXISTS `climate` (
       `id_climate` INT NOT NULL AUTO_INCREMENT,
       `country` VARCHAR(50) DEFAULT NULL,  
       `timepoint` FLOAT DEFAULT NULL, 
       `cloudcover` FLOAT DEFAULT NULL, 
       `highcloud` FLOAT DEFAULT NULL, 
       `midcloud` FLOAT DEFAULT NULL, 
       `lowcloud` FLOAT DEFAULT NULL,
       `temp2m` FLOAT DEFAULT NULL, 
       `lifted_index` FLOAT DEFAULT NULL, 
       `rh2m` FLOAT DEFAULT NULL, 
       `msl_pressure` FLOAT DEFAULT NULL, 
       `prec_amount` FLOAT DEFAULT NULL,
       `snow_depth` FLOAT DEFAULT NULL, 
       `wind_speed` FLOAT DEFAULT NULL, 
       `latitud` FLOAT DEFAULT NULL, 
       `longitud` FLOAT DEFAULT NULL, 
       `rh_layer_950mb` FLOAT DEFAULT NULL,
       `rh_layer_900mb` FLOAT DEFAULT NULL, 
       `rh_layer_850mb` FLOAT DEFAULT NULL, 
       `rh_layer_800mb` FLOAT DEFAULT NULL, 
       `rh_layer_750mb` FLOAT DEFAULT NULL,
       `rh_layer_700mb` FLOAT DEFAULT NULL, 
       `rh_layer_650mb` FLOAT DEFAULT NULL, 
       `rh_layer_600mb` FLOAT DEFAULT NULL, 
       `rh_layer_550mb` FLOAT DEFAULT NULL,
       `rh_layer_500mb` FLOAT DEFAULT NULL,
       `rh_layer_450mb` FLOAT DEFAULT NULL, 
       `rh_layer_400mb` FLOAT DEFAULT NULL, 
       `rh_layer_350mb` FLOAT DEFAULT NULL,
       `rh_layer_300mb` FLOAT DEFAULT NULL,
       `rh_layer_250mb` FLOAT DEFAULT NULL, 
       `rh_layer_200mb` FLOAT DEFAULT NULL,
       `layer950mb_direction` FLOAT DEFAULT NULL, 
       `layer950mb_speed` FLOAT DEFAULT NULL, 
       `layer900mb_direction` FLOAT DEFAULT NULL,
       `layer900mb_speed` FLOAT DEFAULT NULL, 
       `layer850mb_direction` FLOAT DEFAULT NULL, 
       `layer850mb_speed` FLOAT DEFAULT NULL,
       `layer800mb_direction` FLOAT DEFAULT NULL, 
       `layer800mb_speed` FLOAT DEFAULT NULL, 
       `layer750mb_direction` FLOAT DEFAULT NULL,
       `layer750mb_speed` FLOAT DEFAULT NULL, 
       `layer700mb_direction` FLOAT DEFAULT NULL, 
       `layer700mb_speed` FLOAT DEFAULT NULL,
       `layer650mb_direction` FLOAT DEFAULT NULL, 
       `layer650mb_speed` FLOAT DEFAULT NULL,
       `layer600mb_direction` FLOAT DEFAULT NULL,
       `layer600mb_speed` FLOAT DEFAULT NULL,
       `layer550mb_direction` FLOAT DEFAULT NULL, 
       `layer550mb_speed` FLOAT DEFAULT NULL,
       `layer500mb_direction` FLOAT DEFAULT NULL,
       `layer500mb_speed` FLOAT DEFAULT NULL, 
       `layer450mb_direction` FLOAT DEFAULT NULL,
       `layer450mb_speed` FLOAT DEFAULT NULL,
       `layer400mb_direction` FLOAT DEFAULT NULL,
       `layer400mb_speed` FLOAT DEFAULT NULL,
       `layer350mb_direction` FLOAT DEFAULT NULL,
       `layer350mb_speed` FLOAT DEFAULT NULL,
       `layer300mb_direction` FLOAT DEFAULT NULL,
       `layer300mb_speed` FLOAT DEFAULT NULL,
       `layer250mb_direction` FLOAT DEFAULT NULL,
       `layer250mb_speed` FLOAT DEFAULT NULL,
       `layer200mb_direction` FLOAT DEFAULT NULL,
       `layer200mb_speed` FLOAT DEFAULT NULL,
       PRIMARY KEY (`id_climate`))
    
    ENGINE = InnoDB;
       '''

In [30]:
insert_table('tiburones', attacks)

In [31]:
insert_table('tiburones', climate)